## Final Project, Milestone 2

## Milestone 2 Goals from the Proposal:

1.	A data frame of high school media constructed from a python-aided YouTube search.
2.	Maybe a data frame of high school media constructed from a python-aided Google search.

I decided to add merging the two dataframes I created last week to the list of goals for this week. 

## Summary of Milestone 2 Accomplishments:

**Task 1: Data Cleaning** 
1. I cleaned up the large schools dataset, eliminating a number of schools that are not likely to host student media. The number of values in this dataset decreased from 28,112 to 22,398 (20% decrease).
2. In the sno_media dataset, I added media URLs, which had to be extracted from links in the original scraped spreadsheet. I also added an "id" variable, which may be useful when merging the two datasets (more on this below). I changed state names to state abbreviations to match the schools dataset.
3. I renamed the key variables ('School', 'City') to match in both datasets.

**Task 2: Merging the Dataframes**
1. I started by using the merge method. On first pass, the merge matched only 13% of the media to schools in the schools dataframe. When I examined the leftover cases, it became obvious that the merge is case sensitive. When I accounted for differing cases, the merge 27% of the media. This was a good improvement but left 1,880 cases to merge by hand.
2. I looked at 'fuzzy merge' approaches but could not successfully load [fuzzy pandas](https://pypi.org/project/fuzzy-pandas/) or its predecessor [CSV Match](https://github.com/maxharlow/csvmatch/). I did not find anything else that looked as appropriate for my goals as these tools.
3. I began writing a script that iterates through the datasets and matches them on city/state and school names. It uses [Fuzzy Wuzzy](https://pypi.org/project/fuzzywuzzy/) to identify school names that appear close and then prompts for a human decision about whether two school names are, in fact, the same school.
At the time of this writing, the script is not finished. It is missing a last part that will collect the unmatched media and place them at the bottom of the merged dataframe.

**Task 3: Running a python search of YouTube**

I anticipated running python searches of YouTube and Google as the two goals for this milestone. I ran the YouTube search for high school television channels and limited it to the first 10 results. I spent a while wrangling the results JSON into a readable dataframe. 

Scanning through the information in the channel descriptions, it became obvious that there wasn't enough information in these descriptions -- at least not consistently -- to perform a merge with the schools dataframe. Connecting these YouTube results to the main schools dataframe will end up needing to be a manual project. For the time being, I am going to put aside this dimension of the project and focus on the data I already have collected. 

## Milestone 2 Details:

### Task 1: Clean up data from previous week

I cleaned up the schools dataset by deleting schools that are labeled as alternative, special education, virtual, and temporarily closed because these are unlikely to have student media. I renamed key variables to match in both dataframes.

In [136]:
import pandas as pd

In [137]:
schools = pd.read_csv('Public_School_Characteristics_2021-22.csv')
schools

,X,Y,NCESSCH,SURVYEAR,ST_LEAID,LEA_NAME,School,City,State,CHARTER_TEXT,...,HIALF,HI,TRALM,TRALF,TR,WHALM,WHALF,WH,LATCOD,LONCOD
0,-176.640331,51.868164,2.000100e+10,2021-2022,AK-04,Aleutian Region School District,Adak School,Adak,AK,No,...,3.0,5.0,1.0,0.0,1.0,0.0,1.0,1.0,51.868164,-176.640331
1,-154.173260,56.946831,2.004800e+10,2021-2022,AK-28,Kodiak Island Borough School District,Akhiok School,Akhiok,AK,No,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,56.946831,-154.173260
2,-161.448992,60.913883,2.000040e+10,2021-2022,AK-54,Yupiit School District,Akiachak School,Akiachak,AK,No,...,0.0,0.0,2.0,1.0,3.0,0.0,1.0,1.0,60.913883,-161.448992
3,-161.229711,60.907187,2.000040e+10,2021-2022,AK-54,Yupiit School District,Akiak School,Akiak,AK,No,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,60.907187,-161.229711
4,-165.772270,54.134961,2.000070e+10,2021-2022,AK-56,Aleutians East Borough School District,Akutan School,Akutan,AK,No,...,0.0,0.0,1.0,2.0,3.0,0.0,0.0,0.0,54.134961,-165.772270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22393,-104.618595,44.097024,5.606090e+11,2021-2022,WY-2307000,Weston County School District #7,Upton High School,Upton,WY,No,...,1.0,1.0,2.0,0.0,2.0,40.0,32.0,72.0,44.097024,-104.618595
22394,-104.958098,42.059451,5.605090e+11,2021-2022,WY-1601000,Platte County School District #1,Wheatland High School,Wheatland,WY,No,...,17.0,39.0,1.0,1.0,2.0,115.0,112.0,227.0,42.059451,-104.958098
22395,-107.944684,44.009249,5.606240e+11,2021-2022,WY-2201000,Washakie County School District #1,Worland High School,Worland,WY,No,...,57.0,101.0,6.0,2.0,8.0,152.0,137.0,289.0,44.009249,-107.944684
22396,-105.491657,43.756234,5.601470e+11,2021-2022,WY-0301000,Campbell County School District #1,Wright Jr. & Sr. High School,Wright,WY,No,...,10.0,24.0,1.0,3.0,4.0,86.0,74.0,160.0,43.756234,-105.491657


In [138]:
sno_media = pd.read_csv('SNO school media.csv')
sno_media

,id,Media,URL,School,City,State
0,0,Acropolis Times,https://ahsacropolistimes.com/,Athens High School,Athens,AL
1,1,The Hurricane Herald,https://thehurricaneherald.com/,Brookwood High School,Brookwood,AL
2,2,The Buzz,http://chelseabuzz.com/,Chelsea High School,Chelsea,AL
3,3,The Daphne Oracle,https://daphneoracle.org/,Daphne High School,Daphne,AL
4,4,The Pirateer,https://fhspirateer.com/,Fairhope High School,Fairhope,AL
...,...,...,...,...,...,...
2570,2570,The Lariat,https://cheyennecentralhighlariat.org/,Cheyenne Central High School,Cheyenne,WY
2571,2571,Thunderbolt,http://ehsthunderbolt.com/,Cheyenne East High School,Cheyenne,WY
2572,2572,Spartan Mirror,https://triumphspartanmirror.com/,Triumph High School,Cheyenne,WY
2573,2573,The Prowl,https://phsprowl.com/,Powell High School,Powell,WY


### **Task 2a: Merge the two dataframes**

I used the 'outer' option with merge because I wanted to preserve in the resulting dataframe the schools and media that weren't matched. The other 'method' options do not preserve the unmatched cases.

Initially, merge was only able to match 328 cases. There were 22,398 cases in 'schools', and 2,575 in 'sno_media' (12%). The resulting dataframe had 24,645 cases. 

One problem was that merge is case-sensitive. So I lowercased two of the three keys in both dataframes and ran merge again. The resulting dataframe was 24,267. So this time, the merge was able to match 706 cases (27%). 

In [68]:
schools['School_low'] = schools['School'].str.lower()
schools['City_low'] = schools['City'].str.lower()
sno_media['School_low'] = sno_media['School'].str.lower()
sno_media['City_low'] = sno_media['City'].str.lower()

In [25]:
alldata1 = schools.merge(sno_media, how = 'outer', on = ['State', 'City_low', 'School_low'])
alldata1

,X,Y,NCESSCH,SURVYEAR,ST_LEAID,LEA_NAME,School_x,City_x,State,CHARTER_TEXT,...,WH,LATCOD,LONCOD,School_low,City_low,id,Media,URL,School_y,City_y
0,-176.640331,51.868164,2.000100e+10,2021-2022,AK-04,Aleutian Region School District,Adak School,Adak,AK,No,...,1.0,51.868164,-176.640331,adak school,adak,NaN,NaN,NaN,NaN,NaN
1,-154.173260,56.946831,2.004800e+10,2021-2022,AK-28,Kodiak Island Borough School District,Akhiok School,Akhiok,AK,No,...,0.0,56.946831,-154.173260,akhiok school,akhiok,NaN,NaN,NaN,NaN,NaN
2,-161.448992,60.913883,2.000040e+10,2021-2022,AK-54,Yupiit School District,Akiachak School,Akiachak,AK,No,...,1.0,60.913883,-161.448992,akiachak school,akiachak,NaN,NaN,NaN,NaN,NaN
3,-161.229711,60.907187,2.000040e+10,2021-2022,AK-54,Yupiit School District,Akiak School,Akiak,AK,No,...,1.0,60.907187,-161.229711,akiak school,akiak,NaN,NaN,NaN,NaN,NaN
4,-165.772270,54.134961,2.000070e+10,2021-2022,AK-56,Aleutians East Borough School District,Akutan School,Akutan,AK,No,...,0.0,54.134961,-165.772270,akutan school,akutan,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WI,NaN,...,NaN,NaN,NaN,everest high school,weston,2565.0,The Jet,https://dcejet.com/,Everest High School,Weston
24263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WI,NaN,...,NaN,NaN,NaN,whitefish bay high school,whitefish bay,2566.0,The Tower Times,https://wfbtowertimes.com/,Whitefish Bay High School,Whitefish Bay
24264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WY,NaN,...,NaN,NaN,NaN,cheyenne central high school,cheyenne,2570.0,The Lariat,https://cheyennecentralhighlariat.org/,Cheyenne Central High School,Cheyenne
24265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WY,NaN,...,NaN,NaN,NaN,cheyenne east high school,cheyenne,2571.0,Thunderbolt,http://ehsthunderbolt.com/,Cheyenne East High School,Cheyenne


In [8]:
alldata1.to_csv('firstmerge.csv')

### **Task 2b: Trying a Fuzzy Merge Approach**

#### **Fuzzy Pandas**

Source: [https://pypi.org/project/fuzzy-pandas/](https://pypi.org/project/fuzzy-pandas/)

In [16]:
pip install fuzzy-pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "/Users/pbobko/anaconda3/envs/interactivedata/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/Users/pbobko/anaconda3/envs/interactivedata/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Users/pbobko/anaconda3/envs/interactivedata/lib/python3.12/site-packa

This package did not want to install, and neither did its parent CSV Merge. So I was not able to use either package. 

#### **Fuzzy Wuzzy**

Source: [https://pypi.org/project/fuzzywuzzy/](https://pypi.org/project/fuzzywuzzy/)

In [11]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.7/103.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from fuzzywuzzy import fuzz
import numpy as np

#### **Merging Script Using Fuzzy Wuzzy and Human Input**

In [ ]:
# Set up empty columns for values to populate from the sno_media dataframe
alldata2 = pd.DataFrame(schools)
alldata2['id'] = None

alldata2['Media'] = None
# alldata2[['Media']].fillna('')

alldata3['URL'] = None
# alldata3[['URL']].fillna('')

# Interate through the sno_media dataframe and set up city+state variable
for j in sno_media.index:
    citystate_media = ((sno_media['City'][j]).lower(), sno_media['State'][j])

# Iterate through the schools dataframe and set up city+state variable
    for i in schools.index:
        citystate_schools = ((schools['City'][i]).lower(), schools['State'][i])

# Compare the two city+state variables
        if citystate_schools == citystate_media:

# If school names match exactly, add to that row
            if (schools['School'][i]).lower() == (sno_media['School'][j]).lower():

                alldata2.at[i, 'id'] = sno_media['id'][j]
                alldata2.at[i, 'Media'] = sno_media['Media'][j]
                alldata2.at[i, 'URL'] = sno_media['URL'][j]

                print(schools['School'][i], "and", sno_media['School'][j], "merged.")
                print()
                
            # Else if fuzzy comparison reaches some threshold
            else:
                fuzz_score = fuzz.partial_ratio((schools['School'][i]).lower(), (sno_media['School'][j]).lower())
                if fuzz_score >= 75:
                    print("Match score: ", fuzz_score, ". Is ", schools['School'][i], " in ", schools['City'][i], schools['State'][i], " the same as ", sno_media['School'][j], " in ", sno_media['City'][j], sno_media['State'][j], "?")
                    judgment = input()
                    
                    if judgment == "y":
                        alldata2.at[i, 'id'] = sno_media['id'][j]
                        alldata2.at[i, 'Media'] = sno_media['Media'][j]
                        alldata2.at[i, 'URL'] = sno_media['URL'][j]
                        
                        print(schools['School'][i], " and ", sno_media['School'][j], " merged.")
                        print()
                        
                    else:
                        print("Schools not merged.")
                        print()
                
#            else:
#                print("Not a match on school name.")
#                print()
#
#        else:
#            print("Not a match on city and state.")
#            print()

# Make list of ids in schools; for every id not in schools, append row to the bottom of the df
# Extract row from sno_media, make a dataframe, and append it to the bottom of schools
#               row = {'id': sno_media['id'][j], 'Media': sno_media['Media'][j], 'URL': sno_media['URL'][j], 'School': sno_media['School'][j], 'City': sno_media['City'][j], 'State': sno_media['State'][j], 'School_low': sno_media['School_low'][j], 'City_low': sno_media['City_low'][j]}
#               row_df = pd.DataFrame([row])
#               alldata2 = pd.concat([alldata2, row_df], axis=0, ignore_index=True)
#               
                    

### **Task 3: Search YouTube for school media channels**

Source: [https://pypi.org/project/youtube-search-python/](https://pypi.org/project/youtube-search-python/)

In [18]:
pip install youtube-search-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.5/99.5 kB 437.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [19]:
from youtubesearchpython import ChannelsSearch

channelsSearch = ChannelsSearch('high school television', limit = 10, region = 'US')

print(channelsSearch.result())

{'result': [{'type': 'channel', 'id': 'UClGFyu9k3YT0op_KF45IHsA', 'title': 'CTV Charger Television', 'thumbnails': [{'url': '//yt3.googleusercontent.com/ytc/AIdro_mpphwopNhjR7cgk6sMqLR_TpJ_tKrt4kBMaxEgf3Mivhs=s88-c-k-c0x00ffffff-no-rj-mo', 'width': 88, 'height': 88}, {'url': '//yt3.googleusercontent.com/ytc/AIdro_mpphwopNhjR7cgk6sMqLR_TpJ_tKrt4kBMaxEgf3Mivhs=s176-c-k-c0x00ffffff-no-rj-mo', 'width': 176, 'height': 176}], 'videoCount': None, 'descriptionSnippet': None, 'subscribers': '@ctvchargertelevision2635', 'link': 'https://www.youtube.com/channel/UClGFyu9k3YT0op_KF45IHsA'}, {'type': 'channel', 'id': 'UCa6l1maNMDV530iVwWX4Nrg', 'title': 'Viking Television', 'thumbnails': [{'url': '//yt3.ggpht.com/-wibdmBi9F9y-LhmHJZkn3uNgPdsLJumCFD2HTAXhBsqMKrzAuAM93SXVKkVfPQizDIF5kQQQA=s88-c-k-c0x00ffffff-no-rj-mo', 'width': 88, 'height': 88}, {'url': '//yt3.ggpht.com/-wibdmBi9F9y-LhmHJZkn3uNgPdsLJumCFD2HTAXhBsqMKrzAuAM93SXVKkVfPQizDIF5kQQQA=s176-c-k-c0x00ffffff-no-rj-mo', 'width': 176, 'height': 1

In [32]:
type(channelsSearch.result())

dict

In [33]:
pprint.pprint(channelsSearch.result())

{'result': [{'descriptionSnippet': None,
             'id': 'UClGFyu9k3YT0op_KF45IHsA',
             'link': 'https://www.youtube.com/channel/UClGFyu9k3YT0op_KF45IHsA',
             'subscribers': '@ctvchargertelevision2635',
             'thumbnails': [{'height': 88,
                             'url': '//yt3.googleusercontent.com/ytc/AIdro_mpphwopNhjR7cgk6sMqLR_TpJ_tKrt4kBMaxEgf3Mivhs=s88-c-k-c0x00ffffff-no-rj-mo',
                             'width': 88},
                            {'height': 176,
                             'url': '//yt3.googleusercontent.com/ytc/AIdro_mpphwopNhjR7cgk6sMqLR_TpJ_tKrt4kBMaxEgf3Mivhs=s176-c-k-c0x00ffffff-no-rj-mo',
                             'width': 176}],
             'title': 'CTV Charger Television',
             'type': 'channel',
             'videoCount': None},
            {'descriptionSnippet': [{'text': "HF's "},
                                    {'bold': True,
                                     'text': 'High School Television'},
  

In [34]:
search_df = pd.DataFrame(channelsSearch.result())

In [35]:
search_df

,result
0,"{'type': 'channel', 'id': 'UClGFyu9k3YT0op_KF4..."
1,"{'type': 'channel', 'id': 'UCa6l1maNMDV530iVwW..."
2,"{'type': 'channel', 'id': 'UCk746XBgks7S3ASD-O..."
3,"{'type': 'channel', 'id': 'UC3j01Vh7ZO_Q415b1p..."
4,"{'type': 'channel', 'id': 'UCWiBcw_cQK-ajtQEM9..."
5,"{'type': 'channel', 'id': 'UCUYKCd4r8GAnseF-Sm..."
6,"{'type': 'channel', 'id': 'UCsgCEOTFsCxtOjoRpa..."
7,"{'type': 'channel', 'id': 'UCrsBUnY6RqtNxyDvRv..."
8,"{'type': 'channel', 'id': 'UC_otrjRVNx4iJXaMra..."
9,"{'type': 'channel', 'id': 'UC2WTNzwldB2kE0bx88..."


In [73]:
search_df2 = pd.json_normalize(search_df['result'])

In [75]:
search_df2

,type,id,title,thumbnails,videoCount,descriptionSnippet,subscribers,link
0,channel,UClGFyu9k3YT0op_KF45IHsA,CTV Charger Television,[{'url': '//yt3.googleusercontent.com/ytc/AIdr...,None,None,@ctvchargertelevision2635,https://www.youtube.com/channel/UClGFyu9k3YT0o...
1,channel,UCa6l1maNMDV530iVwWX4Nrg,Viking Television,[{'url': '//yt3.ggpht.com/-wibdmBi9F9y-LhmHJZk...,None,"[{'text': 'HF's '}, {'text': 'High School Tele...",@vikingtelevision2170,https://www.youtube.com/channel/UCa6l1maNMDV53...
2,channel,UCk746XBgks7S3ASD-ODs7SQ,Genesee Televison,[{'url': '//yt3.googleusercontent.com/ytc/AIdr...,None,[{'text': 'This is the home of G-TV (Genesee '...,@geneseetelevison1158,https://www.youtube.com/channel/UCk746XBgks7S3...
3,channel,UC3j01Vh7ZO_Q415b1pcdjHA,Lake Central Television,[{'url': '//yt3.ggpht.com/IHVi8MDmQ_zksg7seKIl...,None,"[{'text': 'Lake Central '}, {'text': 'High Sch...",@LakeCentralTelevision,https://www.youtube.com/channel/UC3j01Vh7ZO_Q4...
4,channel,UCWiBcw_cQK-ajtQEM9v8XHQ,IHTN1,[{'url': '//yt3.googleusercontent.com/ytc/AIdr...,None,"[{'text': 'IHTN. Indian Hill '}, {'text': 'Tel...",@IHTN1,https://www.youtube.com/channel/UCWiBcw_cQK-aj...
5,channel,UCUYKCd4r8GAnseF-Sm4D4uw,TPS Television,[{'url': '//yt3.ggpht.com/_Jpkn0gw9-gr_uIryarY...,None,None,@TPSTelevision,https://www.youtube.com/channel/UCUYKCd4r8GAns...
6,channel,UCsgCEOTFsCxtOjoRpatrhhA,WHBSTelevision,[{'url': '//yt3.googleusercontent.com/ytc/AIdr...,None,"[{'text': 'The Westlake '}, {'text': 'High', '...",@WHBSTelevision,https://www.youtube.com/channel/UCsgCEOTFsCxtO...
7,channel,UCrsBUnY6RqtNxyDvRvfUszA,EVHSTelevision,[{'url': '//yt3.googleusercontent.com/ytc/AIdr...,None,"[{'text': 'EVTV is home to Eastview '}, {'text...",@EVHSTelevision,https://www.youtube.com/channel/UCrsBUnY6RqtNx...
8,channel,UC_otrjRVNx4iJXaMraN6jPA,East Wake Television,[{'url': '//yt3.googleusercontent.com/ytc/AIdr...,None,"[{'text': 'We operate two community access '},...",@EastWakeTelevision,https://www.youtube.com/channel/UC_otrjRVNx4iJ...
9,channel,UC2WTNzwldB2kE0bx88Zr-6Q,Auburn High School Television (AHSTV),[{'url': '//yt3.ggpht.com/tguDTrPjlXv_VNhBW0lI...,None,"[{'text': 'Auburn '}, {'text': 'High School Te...",@auburnhighschooltelevisionahst,https://www.youtube.com/channel/UC2WTNzwldB2kE...


In [78]:
search_df2['descriptionSnippet'][1]

[{'text': "HF's "},
 {'text': 'High School Television', 'bold': True},
 {'text': ' Station!'}]

In [79]:
search_df2['descriptionSnippet'][2]

[{'text': 'This is the home of G-TV (Genesee '},
 {'text': 'Television', 'bold': True},
 {'text': ') at Genesee '},
 {'text': 'High School', 'bold': True},
 {'text': ', Genesee, MI. Daily Episodes, Sporting Events, and more\xa0...'}]

In [80]:
search_df2['descriptionSnippet'][3]

[{'text': 'Lake Central '},
 {'text': "High School's", 'bold': True},
 {'text': ' broadcast journalism class.'}]